In [47]:
# import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%run data_cleaning.ipynb

In [48]:
# load the cleaned_dataset
# runs_df = pd.read_csv('run_data.csv')
runs_df.head()

,name,distance_m,moving_time_s,elapsed_time_s,elevation_change_m,type,date,average_cadence,average_heartrate,max_heartrate,pace_per_mile,distance_mi,id,time_of_day,average_zone,max_zone,ratio_avg_hr_to_max_hr
0,Evening Run,5823,0 days 00:30:25,1828,10.9,Run,2022-12-26,86.7,152.4,167,0 days 00:08:24.143646408,3.62,NaN,evening,3,4,0.912575
1,Morning Run,2650,0 days 00:18:30,1633,-6.6,Run,2022-12-28,73.7,133.0,157,0 days 00:11:12.727272727,1.65,NaN,morning,2,4,0.847134
2,Evening Run,2813,0 days 00:14:56,896,23.2,Run,2023-01-08,86.5,160.9,173,0 days 00:08:32,1.75,NaN,evening,4,5,0.930058
3,Afternoon Run,3382,0 days 00:18:22,1179,18.9,Run,2023-02-04,85.9,141.4,176,0 days 00:08:44.761904761,2.10,NaN,afternoon,3,5,0.803409
4,Afternoon Run,3652,0 days 00:22:35,1357,31.8,Run,2023-02-06,82.7,146.8,166,0 days 00:09:56.916299559,2.27,NaN,afternoon,3,4,0.884337


In [49]:
# view trends in the running zone

print(runs_df.groupby('average_zone').size())
print(runs_df.groupby('max_zone').size())

average_zone
2     2
3    84
4    86
dtype: int64
max_zone
3      7
4    111
5     54
dtype: int64


Only 2 of my tracked runs have averaged out to zone 2 running. There is a relatively even split between zone 3 and zone 4 runs though. The zone corresponding to my max heart rate during runs follows a similar trend with only a few runs falling into zone 3 and most falling into zone 4 or 5.

In [53]:
# number of runs in each zone between the two training programs for the colfax half

training_start_2023 = pd.Timestamp(2023, 2, 4).date()
training_end_2023 = pd.Timestamp(2023, 5, 21).date()
training_start_2024 = pd.Timestamp(2024, 1, 21).date()
training_end_2024 = pd.Timestamp(2024, 5, 19).date()

training_1_runs = runs_df.loc[(runs_df['date'] >= training_start_2023) & (runs_df['date'] < training_end_2023)]
training_2_runs = runs_df.loc[(runs_df['date'] >= training_start_2024) & (runs_df['date'] < training_end_2024)]

print(f'2023 Colfax Half Training:\n{training_1_runs.groupby("average_zone").size()}\n')
print(f'2024 Colfax Half Training:\n{training_2_runs.groupby("average_zone").size()}')

2023 Colfax Half Training:
average_zone
3    18
4    21
dtype: int64

2024 Colfax Half Training:
average_zone
3    28
4    20
dtype: int64


In [58]:
average_pace_2023 = training_1_runs['pace_per_mile'].mean()
average_pace_2024 = training_2_runs['pace_per_mile'].mean()
total_miles_2023 = training_1_runs['distance_mi'].sum()
total_miles_2024 = training_2_runs['distance_mi'].sum()

print(f'2023 Colfax Half Training:\nAverage Pace: {average_pace_2023}\nTotal Miles: {total_miles_2023}\nTotal Runs: {training_1_runs.shape[0]}\n')
print(f'2024 Colfax Half Training:\nAverage Pace: {average_pace_2024}\nTotal Miles: {total_miles_2024}\nTotal Runs: {training_2_runs.shape[0]}')

2023 Colfax Half Training:
Average Pace: 0 days 00:08:34.175108744
Total Miles: 152.67000000000002
Total Runs: 39

2024 Colfax Half Training:
Average Pace: 0 days 00:08:39.881966566
Total Miles: 216.37
Total Runs: 48


I ran ~30% less miles in the first training program compared to the second, yet maintained a similar pace. I ran 9 more times leading up to the race which was almost a 25% increase.